In [1]:
# Step 1: Imports and API setup
import os
import openai
from openai import OpenAI
from typing import List, Dict

from dotenv import load_dotenv
load_dotenv()

# Set OpenAI key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# Step 2: Define the user goal
user_goal = "Create a report summarizing the top 3 most talked-about AI startups in July 2025"

In [3]:
client = OpenAI()
def plan_task_with_llm(goal: str) -> str:
    prompt = f"""
You are a task planner agent. Break the following high-level goal into 3-5 clear, ordered subtasks.

Goal: {goal}

Format:
[
  {{ "step": "1", "action": "Describe the task to perform" }},
  ...
]
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful planner agent."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5
    )
    
    content = response.choices[0].message.content
    return content

plan_output = plan_task_with_llm(user_goal)
print(plan_output)

[
  { "step": "1", "action": "Research and identify the top 3 most talked-about AI startups in July 2025 based on news articles, social media trends, and industry reports." },
  { "step": "2", "action": "Gather detailed information on each of the identified startups, including their founding date, key products or services, funding rounds, and notable achievements." },
  { "step": "3", "action": "Analyze the data collected to summarize the significance and impact of each startup within the AI industry during July 2025." },
  { "step": "4", "action": "Draft the report by organizing the information into a clear structure, including an introduction, individual sections for each startup, and a conclusion." },
  { "step": "5", "action": "Review and edit the report for clarity, accuracy, and coherence before finalizing it for distribution." }
]


In [4]:
import json
import re

def parse_plan_output(plan_text: str) -> List[Dict[str, str]]:
    """
    Extracts list of steps from LLM-generated JSON-ish string.
    Falls back to regex line parsing if JSON parsing fails.
    """
    try:
        # Try to parse JSON directly
        parsed = json.loads(plan_text)
        return parsed
    except json.JSONDecodeError:
        # Fallback: naive regex-based extractor
        steps = []
        lines = plan_text.strip().split('\n')
        for line in lines:
            match = re.match(r"^\s*\{?\s*\"?step\"?\s*[:=]?\s*\"?(\d+)\"?.*?\"?action\"?\s*[:=]?\s*\"?(.*?)\"?[,}]?$", line)
            if match:
                steps.append({
                    "step": match.group(1),
                    "action": match.group(2)
                })
            elif re.match(r"^\d+\.", line):  # fallback for "1. Do X"
                step_num, action = line.split(".", 1)
                steps.append({
                    "step": step_num.strip(),
                    "action": action.strip()
                })
        return steps

parsed_steps = parse_plan_output(plan_output)
parsed_steps


[{'step': '1',
  'action': 'Research and identify the top 3 most talked-about AI startups in July 2025 based on news articles, social media trends, and industry reports.'},
 {'step': '2',
  'action': 'Gather detailed information on each of the identified startups, including their founding date, key products or services, funding rounds, and notable achievements.'},
 {'step': '3',
  'action': 'Analyze the data collected to summarize the significance and impact of each startup within the AI industry during July 2025.'},
 {'step': '4',
  'action': 'Draft the report by organizing the information into a clear structure, including an introduction, individual sections for each startup, and a conclusion.'},
 {'step': '5',
  'action': 'Review and edit the report for clarity, accuracy, and coherence before finalizing it for distribution.'}]

In [5]:
# Step 5: Tool Registry + Stub Functions

def search_trending_startups() -> str:
    # Simulated web search tool
    return """
    1. SynthMind – building low-latency multi-modal AI chips.
    2. QuantaFlow – offers self-healing infrastructure for LLMs.
    3. NeuralForge – language agents that build software end-to-end.
    """

# def summarize_startups(raw_text: str) -> str:
#     # Simulated summarizer tool
#     return (
#         "1. **SynthMind**: Leading the way in AI-specific hardware for mobile and edge devices.\n"
#         "2. **QuantaFlow**: Their infra stack auto-scales and auto-heals for real-time LLM needs.\n"
#         "3. **NeuralForge**: Pioneering autonomous coding agents that complete real-world apps."
#     )


def summarize_startups(raw_text: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a startup analyst who summarizes startup news for executive briefings."},
            {"role": "user", "content": f"Summarize the following startup news in 3 points:\n\n{raw_text}"}
        ],
        temperature=0.6
    )
    return response.choices[0].message.content.strip()



def write_markdown(summary: str) -> str:
    # Simulated file writer
    filename = "ai_startup_report.md"
    with open(filename, "w") as f:
        f.write("# Top AI Startups – July 2025\n\n")
        f.write(summary)
    return f"Report written to `{filename}`"

# Tool registry maps keyword to functions
TOOL_REGISTRY = {
    "search": search_trending_startups,
    "summarize": summarize_startups,
    "write": write_markdown
}


In [6]:
# Step 6: Execution Loop

def pick_tool(action_text: str):
    """Naively match tool based on keywords in action."""
    action_lower = action_text.lower()
    if "search" in action_lower or "find" in action_lower:
        return "search"
    elif "summarize" in action_lower or "extract" in action_lower:
        return "summarize"
    elif "write" in action_lower or "report" in action_lower:
        return "write"
    else:
        return None

# Memory to pass output between tools
intermediate_memory = None

# Execute each step
for step in parsed_steps:
    step_id = step["step"]
    action = step["action"]
    print(f"\n🧭 Step {step_id}: {action}")

    tool_key = pick_tool(action)
    if not tool_key:
        print("❌ No matching tool found.")
        continue

    tool_fn = TOOL_REGISTRY.get(tool_key)
    if not tool_fn:
        print("⚠️ Tool function not implemented.")
        continue

    # Decide input based on tool type
    if tool_key in ["search"]:
        result = tool_fn()
    else:
        result = tool_fn(intermediate_memory)

    intermediate_memory = result  # Save result for next step
    print("✅ Output:\n", result)



🧭 Step 1: Research and identify the top 3 most talked-about AI startups in July 2025 based on news articles, social media trends, and industry reports.
✅ Output:
 
    1. SynthMind – building low-latency multi-modal AI chips.
    2. QuantaFlow – offers self-healing infrastructure for LLMs.
    3. NeuralForge – language agents that build software end-to-end.
    

🧭 Step 2: Gather detailed information on each of the identified startups, including their founding date, key products or services, funding rounds, and notable achievements.
❌ No matching tool found.

🧭 Step 3: Analyze the data collected to summarize the significance and impact of each startup within the AI industry during July 2025.
✅ Output:
 1. **SynthMind** is developing low-latency multi-modal AI chips aimed at enhancing the performance and efficiency of AI applications across various domains.

2. **QuantaFlow** provides self-healing infrastructure specifically designed for large language models (LLMs), ensuring resilienc